In [ ]:
#!/usr/bin/env python
# coding: utf-8

import shioaji as sj
import pandas as pd
import os

# 設定命令列參數
import argparse
parser = argparse.ArgumentParser(description="Generate stock data pickle")
parser.add_argument("--stock", type=str, required=True, help="Stock symbol")
parser.add_argument("--date", type=str, required=True, help="Date in YYYYMMDD format")
args = parser.parse_args()

# 確保日期格式為 YYYY-MM-DD
args.date = f"{args.date[:4]}-{args.date[4:6]}-{args.date[6:]}"
print(f"日期已轉換為: {args.date}")

# 登入 Shioaji
print("正在登入 Shioaji...")
api = sj.Shioaji()
accounts = api.login("Csnb96YobsMsZDCx2LH1fb1kmrbPTiqP3DXh32mJjJ5H", "F5vogGkcQ3TAZ2TqPpaCHQ9YuxoeoeQwZpqw4jSC94mM")
print("登入成功")

# 第二次登入
api.login(
    api_key="Csnb96YobsMsZDCx2LH1fb1kmrbPTiqP3DXh32mJjJ5H",
    secret_key="F5vogGkcQ3TAZ2TqPpaCHQ9YuxoeoeQwZpqw4jSC94mM",
    contracts_cb=lambda security_type: print(f"{repr(security_type)} fetch done.")
)

# 檢查 CA 憑證是否存在
if os.path.exists("C:/Users/user/Desktop/20240911_2330/Sinopac.pfx"):
    print("找到 CA 憑證，正在激活...")
    result = api.activate_ca(
        ca_path="C:/Users/user/Desktop/20240911_2330/Sinopac.pfx",  # 憑證路徑
        ca_passwd="A126852603",  # 憑證密碼
        person_id="A126852603",  # 身分證字號
    )
    print("CA 憑證成功")
else:
    print("CA 憑證不存在，請檢查路徑！")
    exit(1)

# 取得股票資料
print(f"正在取得 {args.stock} 的股票資料...")
ticks = api.ticks(
    contract=api.Contracts.Stocks[args.stock],
    date=args.date
)
print(f"{args.stock} 的股票資料取得成功")

# 將 ticks 資料轉換為 DataFrame
print("正在將 ticks 資料轉換為 DataFrame...")
df_ticks = pd.DataFrame({**ticks})
df_ticks['ts'] = pd.to_datetime(df_ticks['ts'])

# 取得股票的快照資料
print(f"正在取得 {args.stock} 的股票快照資料...")
contracts = [api.Contracts.Stocks[args.stock]]
snapshots = api.snapshots(contracts)

# 將快照轉換為 DataFrame
print("正在將快照資料轉換為 DataFrame...")
df_snapshots = pd.DataFrame(s.__dict__ for s in snapshots)
df_snapshots['ts'] = pd.to_datetime(df_snapshots['ts'])

# 擷取需要的欄位
cols_to_extract = ['ts', 'open', 'high', 'low', 'close', 'volume']
df_selected = df_snapshots[cols_to_extract]
print("成功擷取所需欄位")

# 獲取 main.py 所在的資料夾路徑
save_dir = os.path.dirname(os.path.abspath(__file__))

# 保存至 main.py 所在的資料夾
filename = f"{args.date}_{args.stock}.pickle"
save_path = os.path.join(save_dir, filename)
df_selected.to_pickle(save_path)
print(f"資料已保存至 {save_path}")

# 登出 Shioaji
api.logout()
print("已成功登出 Shioaji")
